In [ ]:
import numpy as np 
import pandas as pd 
import requests, xlsxwriter, math
from scipy import stats 
stocks = pd.read_csv('stocks.csv')
IEX_CLOUD_API_TOKEN = ###########
symbol = 'SPY'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()

In [ ]:

def groups(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(groups(stocks['Symbol'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))

my_columns = ['Symbol', 'Price', '1Y Price Return', 'Shares to Buy'] 

In [ ]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A'
                                                   ], 
                                                  index = my_columns), 
                                        ignore_index = True)
final_dataframe.sort_values('1Y Price Return', ascending = False, inplace = True)
final_dataframe = final_dataframe[:51]
final_dataframe.reset_index(drop = True, inplace = True)


In [ ]:
def portfolio_size():
    global portfolio_size
    portfolio_size = input("What's your portfolio size:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("portfolio size can be either float or int:")
        portfolio_size = input("What's your portfolio size:")

portfolio_size()

In [ ]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Symbol'])):
    final_dataframe.loc[i, 'Shares to buy'] = math.floor(position_size / final_dataframe['Price'][i])

In [ ]:
hqm_columns = [
                'Symbol', 
                'Price', 
                'Shares to buy', 
                '1Y Price Return', 
                '1Y Return Percentile',
                '6M Price Return',
                '6M Return Percentile',
                '3 Months Price Return',
                '3M Return Percentile',
                '1 Month Price Return',
                '1M Return Percentile',
                'HQM Score'
                ]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   'N/A',
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month6ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month3ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month1ChangePercent'],
                                                   'N/A',
                                                   'N/A'
                                                   ], 
                                                  index = hqm_columns), 
                                        ignore_index = True)
        
hqm_dataframe.columns

In [ ]:
time_periods = [
                '1 Year',
                '6 Months',
                '3 Months',
                '1 Month'
                ]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        hqm_dataframe.loc[row, f'{time_period} Return Percentile'] = stats.percentileofscore(hqm_dataframe[f'{time_period} Price Return'], hqm_dataframe.loc[row, f'{time_period} Price Return'])/100

for time_period in time_periods:
    print(hqm_dataframe[f'{time_period} Return Percentile'])    
hqm_dataframe

In [ ]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)

In [ ]:
hqm_dataframe.sort_values(by = 'HQM Score', ascending = False)
hqm_dataframe = hqm_dataframe[:51]

In [ ]:
portfolio_size()

In [ ]:
position_size = float(portfolio_size) / len(hqm_dataframe.index)
for i in range(0, len(hqm_dataframe['Symbol'])-1):
    hqm_dataframe.loc[i, 'Shares to buy'] = math.floor(position_size / hqm_dataframe['Price'][i])
hqm_dataframe

In [ ]:
writer = pd.ExcelWriter('momentum_trading.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name='Momentum Trading', index = False)

In [ ]:
background_color = '#9f9fc2'
font_color = '#786c6c'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [ ]:
column_formats = { 
                    'A': ['Symbol', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Shares to buy', integer_template],
                    'D': ['1Y Price Return', percent_template],
                    'E': ['1Y Return Percentile', percent_template],
                    'F': ['6M Price Return', percent_template],
                    'G': ['6M Return Percentile', percent_template],
                    'H': ['3 Months Price Return', percent_template],
                    'I': ['3M Return Percentile', percent_template],
                    'J': ['1 Month Price Return', percent_template],
                    'K': ['1M Return Percentile', percent_template],
                    'L': ['HQM Score', integer_template]
                    }

for column in column_formats.keys():
    writer.sheets['Momentum Trading'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Momentum Trading'].write(f'{column}1', column_formats[column][0], string_template)

In [ ]:
writer.save()